# 英翻中、中翻英 LLM 應用程式
輸入 英文, LLM 產生中文翻譯, 然後將翻譯結果印出來

輸入 中文, LLM 產生英文翻譯, 然後將翻譯結果印出來

## Define the task dispatcher chain



In [1]:
# load the API key
from dotenv import load_dotenv

load_dotenv()

True

In [22]:
task_dispatcher_prompt = """
You are a task dispatcher. You decide what task to do based on the input language.

You first check the what kind of language the input is, English or Chinese, or other.
Then,  You will return a JSON object **with no additional explanation** with the following fields:
- "key": the task key
- "input": the user's input

<input>
{user_input}

<Available tasks (task_key:rules)>
- `eng`: When the input is in Chinese, translate it to English.
- `chinese`: When the input is in English, translate it to Chinese.
- `default`: For other cases, it's not supported language.
"""


In [2]:
# Create the chat model
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [23]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

# Create the dispatcher runnable
from langchain_core.prompts import PromptTemplate


dispatcher_runnable = (
    PromptTemplate.from_template(task_dispatcher_prompt)
    | llm
    | JsonOutputParser()
)

In [12]:
# Test the dispatcher runnable
resp_1 = dispatcher_runnable.invoke({"user_input": "今天天氣真好"})
print(resp_1)
resp_2 = dispatcher_runnable.invoke({"user_input": "The weather is nice today"})
print(resp_2)
resp_3 = dispatcher_runnable.invoke({"user_input": "Bonjour"})
print(resp_3)

{'key': 'eng', 'input': '今天天氣真好'}
{'key': 'chinese', 'input': 'The weather is nice today'}
{'key': 'default', 'input': 'Bonjour'}


## Define the ch-eng translation chain

In [7]:
# Create the Chinese to English translation chain
ch_to_eng_prompt_str = """
You are a professional Chinese-to-English translator and editor.

Your task is:
1. First, **polish** the provided Traditional Chinese input to make it clearer, more fluent, and natural in tone.
2. Then, **translate** the polished version into fluent, professional English.

Output only in the following format, with no additional comments or explanation:
- [polished_chinese]: <your polished Chinese version>
- [english_translation]: <your English translation>

Here is the original input:
{user_input}
"""

# create the ch_to_eng runnable
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

ch_to_eng_prompt = PromptTemplate.from_template(ch_to_eng_prompt_str)
print(ch_to_eng_prompt.invoke({"user_input": "今天天氣真好"}))
ch_to_eng_runnable = (
    ch_to_eng_prompt | llm | StrOutputParser()
)


text='\nYou are a professional Chinese-to-English translator and editor.\n\nYour task is:\n1. First, **polish** the provided Traditional Chinese input to make it clearer, more fluent, and natural in tone.\n2. Then, **translate** the polished version into fluent, professional English.\n\nOutput only in the following format, with no additional comments or explanation:\n- [polished_chinese]: <your polished Chinese version>\n- [english_translation]: <your English translation>\n\nHere is the original input:\n今天天氣真好\n'


In [8]:
# Test the Chinese to English translation chain
chinese_str = "今天天氣真好"
resp_4 = ch_to_eng_runnable.invoke({"user_input": chinese_str})
print(resp_4)

- [polished_chinese]: 今天天氣真好，陽光明媚。
- [english_translation]: The weather is really nice today, with bright sunshine.


## Define the eng-ch translation chain

In [14]:
eng_to_ch_prompt_str = """
You are a professional English-to-Chinese translator and editor.

Your task is:
1. First, **polish** the provided English input to make it clearer, more fluent, and natural in tone.
2. Then, **translate** the polished version into fluent, professional Traditional Chinese.

Output only in the following format, with no additional comments or explanation:
- [polished_english]: <your polished English version>
- [chinese_translation]: <your Chinese translation>

Here is the original input:
{user_input}
"""


In [16]:
# Create the English to Chinese translation chain
eng_to_ch_prompt = PromptTemplate.from_template(eng_to_ch_prompt_str)
eng_to_ch_runnable = ( 
    eng_to_ch_prompt | llm | StrOutputParser()
)

In [17]:
# Test the English to Chinese translation chain
english_str = "The weather is nice today"
resp_5 = eng_to_ch_runnable.invoke({"user_input": english_str})
print(resp_5)


- [polished_english]: The weather is beautiful today.
- [chinese_translation]: 今天天氣很好。


## Define the RouterRunnable

In [28]:
from langchain_core.runnables import RouterRunnable, RunnableLambda

## Create the RouterRunnable

router_runnable = RouterRunnable(
    runnables={
        "eng": ch_to_eng_runnable,
        "chinese": eng_to_ch_runnable,
        "default": RunnableLambda(lambda x: "Not supported language (不支援的語言)")
        })




# Create the main chain

In [29]:
translate_chain = (
    dispatcher_runnable
    | router_runnable
    | StrOutputParser()
)


In [30]:
# Test the main chain
chinese_str = "今天天氣真好"
english_str = "The weather is nice today"
french_str = "Bonjour! Comment ça va?"
print("Chinese to English translation:")
resp_6 = translate_chain.invoke({"user_input": chinese_str})
print(resp_6)
print("English to Chinese translation:")
resp_7 = translate_chain.invoke({"user_input": english_str})
print(resp_7)
print("French to English translation:")
resp_8 = translate_chain.invoke({"user_input": french_str})
print(resp_8)


Chinese to English translation:
- [polished_chinese]: 今天天氣非常好。
- [english_translation]: The weather today is really nice.
English to Chinese translation:
- [polished_english]: The weather is beautiful today.
- [chinese_translation]: 今天的天氣真好。
French to English translation:
Not supported language (不支援的語言)


In [31]:
# Print the chain ascii tree
translate_chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
  +------------------+     
  | JsonOutputParser |     
  +------------------+     
            *              
            *              
            *              
    +----------------+     
    | RouterRunnable |     
    +----------------+     
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *       